## 모듈 임포트, 데이터프레임 불러오기

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

import statsmodels.formula.api as smf
import statsmodels.api as sm

import xgboost as xgb
from xgboost import XGBRegressor

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [2]:
train_ = pd.read_csv('dataset/train.csv')
test_ = pd.read_csv('dataset/test.csv')

train_.head()

,id,Store,Date,Temperature,Fuel_Price,Promotion1,Promotion2,Promotion3,Promotion4,Promotion5,Unemployment,IsHoliday,Weekly_Sales
0,1,1,05/02/2010,42.31,2.572,NaN,NaN,NaN,NaN,NaN,8.106,False,1643690.90
1,2,1,12/02/2010,38.51,2.548,NaN,NaN,NaN,NaN,NaN,8.106,True,1641957.44
2,3,1,19/02/2010,39.93,2.514,NaN,NaN,NaN,NaN,NaN,8.106,False,1611968.17
3,4,1,26/02/2010,46.63,2.561,NaN,NaN,NaN,NaN,NaN,8.106,False,1409727.59
4,5,1,05/03/2010,46.50,2.625,NaN,NaN,NaN,NaN,NaN,8.106,False,1554806.68


## 전처리 함수 생성(연, 월, 일 처리)

In [3]:
# 편한 가독성을 위해 연, 월, 일을 분리

def split_date(df):
    y = []
    m = []
    d = []
    
    for i in df['Date']:
        d_, m_, y_ = i.split('/')
        y.append(int(y_))
        m.append(int(m_))
        d.append(int(d_))
        
    df.insert(3, 'Y', y)
    df.insert(4, 'M', m)
    df.insert(5, 'D', d)
    
    return df

# holiday이면 1, 아니면 0

def holiday(df):
    tmp = []
    
    for i in df['IsHoliday']:
        if i == False:
            tmp.append(0)
        else:
            tmp.append(1)
            
    df.insert(15, 'Holiday', tmp)
    
    return df

# 월과 일을 합침

def only_date(df):
    # day of year
    doy = []
    
    for i in zip(df['Y'], df['M'], df['D']):
        period = pd.Period("{}-{}-{}".format(i[0], i[1], i[2]))
        doy.append(period.day_of_year)
        
    df.insert(6, 'DOY', doy)
        
    return df

In [4]:
train_ = split_date(train_)
train_ = holiday(train_)
train_ = only_date(train_)

train_ = train_.fillna(0)

test_ = split_date(test_)
test_ = holiday(test_)
test_ = only_date(test_)

test_ = test_.fillna(0)

train_.head()

,id,Store,Date,Y,M,D,DOY,Temperature,Fuel_Price,Promotion1,Promotion2,Promotion3,Promotion4,Promotion5,Unemployment,IsHoliday,Holiday,Weekly_Sales
0,1,1,05/02/2010,2010,2,5,36,42.31,2.572,0.0,0.0,0.0,0.0,0.0,8.106,False,0,1643690.90
1,2,1,12/02/2010,2010,2,12,43,38.51,2.548,0.0,0.0,0.0,0.0,0.0,8.106,True,1,1641957.44
2,3,1,19/02/2010,2010,2,19,50,39.93,2.514,0.0,0.0,0.0,0.0,0.0,8.106,False,0,1611968.17
3,4,1,26/02/2010,2010,2,26,57,46.63,2.561,0.0,0.0,0.0,0.0,0.0,8.106,False,0,1409727.59
4,5,1,05/03/2010,2010,3,5,64,46.50,2.625,0.0,0.0,0.0,0.0,0.0,8.106,False,0,1554806.68


In [5]:
test_.head()

,id,Store,Date,Y,M,D,DOY,Temperature,Fuel_Price,Promotion1,Promotion2,Promotion3,Promotion4,Promotion5,Unemployment,IsHoliday,Holiday
0,1,1,05/10/2012,2012,10,5,279,68.55,3.617,8077.89,0.00,18.22,3617.43,3626.14,6.573,False,0
1,2,1,12/10/2012,2012,10,12,286,62.99,3.601,2086.18,0.00,8.11,602.36,5926.45,6.573,False,0
2,3,1,19/10/2012,2012,10,19,293,67.97,3.594,950.33,0.00,4.93,80.25,2312.85,6.573,False,0
3,4,1,26/10/2012,2012,10,26,300,69.16,3.506,2585.85,31.75,6.00,1057.16,1305.01,6.573,False,0
4,5,2,05/10/2012,2012,10,5,279,70.27,3.617,6037.76,0.00,10.04,3027.37,3853.40,6.170,False,0


## Promotion 2와 4를 포함시키지 않을 지점 추려내기

In [6]:
l1 = []
l2 = []

for id_ in range(1, 46):
    store_ = train_[(train_['Store'] == id_) & (train_['Promotion1'] != 0.0)]
    
    # 나누는 기준 : Promotion 2, 3, 4에서 1/3 이상이 결측치인 경우
    count = (store_['Promotion2'] != 0.0).sum() + (store_['Promotion3'] != 0.0).sum() + (store_['Promotion4'] != 0.0).sum()
    if count >= len(store_) * 2:
        l1.append(id_)
    else:
        l2.append(id_)
        
print(l1)
print(l2)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 31, 32, 34, 35, 39, 40, 41, 45]
[30, 33, 36, 37, 38, 42, 43, 44]


## Standard Scaling 실행, 필요 없는 행 제거

In [7]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

features = train_.iloc[:, 9:14]
colnames = features.columns + ['_Scaled']

features_scaled = pd.DataFrame(scaler.fit_transform(features), columns = colnames)

train_standard = pd.concat([train_, features_scaled], axis = 1)
train_standard.drop(['Promotion1', 'Promotion2', 'Promotion3', 'Promotion4', 'Promotion5', 
                     'M', 'D', 'Temperature', 'Unemployment', 'id', 'Date', 'IsHoliday'], axis = 1, inplace = True)

features = test_.iloc[:, 8:13]
colnames = features.columns + ['_Scaled']

features_scaled = pd.DataFrame(scaler.fit_transform(features), columns = colnames)

test_standard = pd.concat([test_, features_scaled], axis = 1)
test_standard.drop(['Promotion1', 'Promotion2', 'Promotion3', 'Promotion4', 'Promotion5', 
                    'M', 'D', 'Temperature', 'Unemployment', 'id', 'Date', 'IsHoliday'], axis = 1, inplace = True)

In [8]:
train_standard

,Store,Y,DOY,Fuel_Price,Holiday,Weekly_Sales,Promotion1_Scaled,Promotion2_Scaled,Promotion3_Scaled,Promotion4_Scaled,Promotion5_Scaled
0,1,2010,36,2.572,0,1643690.90,-0.405482,-0.170211,-0.082414,-0.265323,-0.371303
1,1,2010,43,2.548,1,1641957.44,-0.405482,-0.170211,-0.082414,-0.265323,-0.371303
2,1,2010,50,2.514,0,1611968.17,-0.405482,-0.170211,-0.082414,-0.265323,-0.371303
3,1,2010,57,2.561,0,1409727.59,-0.405482,-0.170211,-0.082414,-0.265323,-0.371303
4,1,2010,64,2.625,0,1554806.68,-0.405482,-0.170211,-0.082414,-0.265323,-0.371303
...,...,...,...,...,...,...,...,...,...,...,...
6250,45,2012,244,3.867,0,734297.87,3.580831,-0.168998,-0.065037,1.574261,0.598779
6251,45,2012,251,3.911,1,766512.66,1.453422,-0.167623,-0.072573,0.222921,0.128229
6252,45,2012,258,3.948,0,702238.27,1.518086,-0.170211,-0.081610,0.635401,0.909037
6253,45,2012,265,4.038,0,723086.20,1.019698,-0.151556,-0.070589,0.360229,1.735591


In [9]:
test_standard

,Store,Y,DOY,Fuel_Price,Holiday,Fuel_Price_Scaled,Promotion1_Scaled,Promotion2_Scaled,Promotion3_Scaled,Promotion4_Scaled
0,1,2012,279,3.617,0,-0.863006,1.576197,-0.368901,-0.198291,1.726332
1,1,2012,286,3.601,0,-0.921425,-0.388945,-0.368901,-0.418291,-0.294492
2,1,2012,293,3.594,0,-0.946984,-0.761478,-0.368901,-0.487490,-0.644432
3,1,2012,300,3.506,0,-1.268291,-0.225065,0.355530,-0.464206,0.010333
4,2,2012,279,3.617,0,-0.863006,0.907081,-0.368901,-0.376293,1.330850
...,...,...,...,...,...,...,...,...,...,...
175,44,2012,300,3.755,0,-0.359138,-1.060488,-0.309349,-0.573445,-0.698219
176,45,2012,279,3.985,0,0.480642,0.582050,-0.368901,-0.185234,0.812123
177,45,2012,286,4.000,0,0.535410,-0.431550,-0.368901,-0.423079,-0.296530
178,45,2012,293,3.969,0,0.422223,-0.415892,-0.368901,-0.525572,-0.404834


## 예측 실행, 최종 제출 파일 생성

In [10]:
prediction_list = []

for id_ in range(1, 46):
    store_train = train_standard[train_standard['Store'] == id_]
    store_test = test_standard[test_standard['Store'] == id_]
    
    if id_ in l1:
        X = store_train.drop('Weekly_Sales', axis = 1)
        x = store_test
    else:
        X = store_train.drop(['Weekly_Sales', 'Promotion2_Scaled', 'Promotion4_Scaled'], axis = 1)
        x = store_test.drop(['Promotion2_Scaled', 'Promotion4_Scaled'], axis = 1)
        
    Y = store_train['Weekly_Sales']
    model = XGBRegressor(n_estimators = 100, learning_rate = 0.1,
                             min_child_weight = 2, max_depth = 6)
    
    model.fit(X, Y)
    
    y = model.predict(x)
    prediction_list.append(y)

In [11]:
prediction = []

for i in prediction_list:
    for values in i:
        prediction.append(values)

In [12]:
df = pd.DataFrame({'id' : list(range(1, len(test_standard) + 1)), 'Weekly_Sales' : prediction})
df.set_index('id', inplace = True)
df

,Weekly_Sales
id,
1,1.502958e+06
2,1.505236e+06
3,1.502322e+06
4,1.621916e+06
5,1.777757e+06
...,...
176,3.147400e+05
177,7.330093e+05
178,7.332999e+05


In [13]:
df.to_csv('./최종 예측.csv', encoding = 'utf-8-sig')